In [1]:
import csv
import json
import pandas as pd

In [45]:
related_matches = pd.read_csv('related_matches.csv')

In [46]:
related_matches

,label_uri,literal,lang,query_terms,aat,wikidata,pwn,odwn,nmvw
0,l_01,Aboriginal,en,aboriginal#aboriginals,300379660,Q103817,native.n.01#native.a.03,None,None
1,l_07,Allochtoon,en,allochtoon#allochtoons,None,Q2008347,None,None,None
2,l_100,Baboo,en,baboo#baboos,None,None,None,None,None
3,l_10,Barbarian,en,barbarian#barbarians,None,Q134313,savage.n.01#peasant.n.03#barbarian.s.01,None,None
4,l_21,Batavia,en,batavia#batavias,None,Q1199713,None,None,None
...,...,...,...,...,...,...,...,...,...
153,l_282,Westers,nl,westers#wester#westerse#westersere#westerste#w...,None,Q160381,None,None,None
154,l_200,Wit,nl,wit#witten#wittere#witst#witte#witter#wits#witste,None,Q235155,None,None,None
155,l_283,Zigeuner,nl,zigeuner#zigeuners,300404070,Q76475,None,zigeuner-n-1,10075876
156,l_169,Zuid­-Rhodesië,nl,zuid-rhodesië,None,Q750583,None,None,None


In [88]:
# generating a dict
related_matches_dict = {}

for row in related_matches.iterrows():
    
    rm_per_label = {'aat':[row[1].aat],
                    'wikidata':[row[1].wikidata],
                    'pwn':row[1].pwn.split('#'),
                    'odwn':row[1].odwn.split('#'),
                    'nmvw':[row[1].nmvw]}
    
    related_matches_dict[row[1].label_uri] = {'literal_form':row[1].literal,
                                             'lang':row[1].lang,
                                             'query_terms':row[1].query_terms.split('#'),
                                             'related_matches':rm_per_label}

In [89]:
related_matches_dict

{'l_01': {'literal_form': 'Aboriginal',
  'lang': 'en',
  'query_terms': ['aboriginal', 'aboriginals'],
  'related_matches': {'aat': ['300379660'],
   'wikidata': ['Q103817'],
   'pwn': ['native.n.01', 'native.a.03'],
   'odwn': ['None'],
   'nmvw': ['None']}},
 'l_07': {'literal_form': 'Allochtoon',
  'lang': 'en',
  'query_terms': ['allochtoon', 'allochtoons'],
  'related_matches': {'aat': ['None'],
   'wikidata': ['Q2008347'],
   'pwn': ['None'],
   'odwn': ['None'],
   'nmvw': ['None']}},
 'l_100': {'literal_form': 'Baboo',
  'lang': 'en',
  'query_terms': ['baboo', 'baboos'],
  'related_matches': {'aat': ['None'],
   'wikidata': ['None'],
   'pwn': ['None'],
   'odwn': ['None'],
   'nmvw': ['None']}},
 'l_10': {'literal_form': 'Barbarian',
  'lang': 'en',
  'query_terms': ['barbarian', 'barbarians'],
  'related_matches': {'aat': ['None'],
   'wikidata': ['Q134313'],
   'pwn': ['savage.n.01', 'peasant.n.03', 'barbarian.s.01'],
   'odwn': ['None'],
   'nmvw': ['None']}},
 'l_21': {'

In [44]:
# saving realted matches as json
with open('related_matches.json', 'w') as jf:
    json.dump(related_matches_dict, jf)

### Merging related matches with literals

#### Importing literals from the json files
 the files were generated with lodlitparser.get_literals()

In [90]:
# AAT EN
path_aat_en = 'https://github.com/cultural-ai/wordsmatter/raw/main/rm.json'
related_matches = requests.get(path_to_rm).json()

with open('/Users/anesterov/reps/LODlit/aat_rm_en.json','r') as jf:
    aat_en = json.load(jf)
# AAT NL
with open('/Users/anesterov/reps/LODlit/aat_rm_nl.json','r') as jf:
    aat_nl = json.load(jf)
# Wikidata EN
with open('/Users/anesterov/reps/LODlit/wikidata_rm_en.json','r') as jf:
    wikidata_en = json.load(jf)
# Wikidata NL
with open('/Users/anesterov/reps/LODlit/wikidata_rm_nl.json','r') as jf:
    wikidata_nl = json.load(jf)
# PWN
with open('/Users/anesterov/reps/LODlit/pwn_rm.json','r') as jf:
    pwn_rm = json.load(jf)
# ODWN
with open('/Users/anesterov/reps/LODlit/odwn_rm.json','r') as jf:
    odwn_rm = json.load(jf)
# NMVW
with open('/Users/anesterov/reps/LODlit/nmvw_rm.json','r') as jf:
    nmvw_rm = json.load(jf)

In [91]:
# this code updates related_matches_dict
for label_uri, values in related_matches_dict.items():
    
    literals_per_label = {}
    
    # EN
    if values['lang'] == 'en':
        # aat
        per_resource =[]
        if values['related_matches']['aat'][0] != 'None':
            aat_uri = values['related_matches']['aat'][0]
            aat_en[aat_uri]['aat_uri'] = aat_uri
            per_resource.append(aat_en[aat_uri])

        literals_per_label['aat_en'] = per_resource
        
        # wikidata
        per_resource =[]
        if values['related_matches']['wikidata'][0] != 'None':
            wikidata_q = values['related_matches']['wikidata'][0]
            for result in wikidata_en:
                if wikidata_q in result.keys():
                    per_resource.append(result[wikidata_q])

        literals_per_label['wikidata_en'] = per_resource
        
        # PWN
        per_resource = []
        for synset_id in values['related_matches']['pwn']:
            if synset_id != 'None':
                pwn_rm[synset_id]['synset_id'] = synset_id
                dict_per_hit = pwn_rm[synset_id]
                per_resource.append(dict_per_hit)
            
        literals_per_label['pwn'] = per_resource
        
    # NL
    if values['lang'] == 'nl':
        # aat
        per_resource = []
        if values['related_matches']['aat'][0] != 'None':
            aat_uri = values['related_matches']['aat'][0]
            aat_nl[aat_uri]['aat_uri'] = aat_uri
            per_resource.append(aat_nl[aat_uri])

        literals_per_label['aat_nl'] = per_resource
        
        #odwn
        per_resource = []
        for le_id in values['related_matches']['odwn']:
            if le_id != 'None':
                odwn_rm[le_id]['LE_id'] = le_id
                dict_per_hit = odwn_rm[le_id]
                per_resource.append(dict_per_hit)
                
        literals_per_label['odwn'] = per_resource
        
        #nmvw
        per_resource = []
        if values['related_matches']['nmvw'][0] != 'None':
            nmvw_handle = f"https://hdl.handle.net/20.500.11840/termmaster{values['related_matches']['nmvw'][0]}"
            nmvw_rm[nmvw_handle]['nmvw_handle'] = nmvw_handle
            per_resource.append(nmvw_rm[nmvw_handle])
        
        literals_per_label['nmvw'] = per_resource
        
    values['related_matches_literals'] = literals_per_label

In [93]:
with open('rm_literals.json', 'w') as jf:
    json.dump(related_matches_dict, jf)

In [94]:
aat_en

{'300379660': {'lang': 'en',
  'prefLabel': 'indigenous people',
  'altLabels': ['indigenous person',
   'indigenous peoples',
   'aboriginal peoples',
   'indigenous populations',
   'native peoples',
   'aborigines',
   'indigenes',
   'people, indigenous'],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': 'The original, or earliest known, inhabitants of a colony or country.',
  'aat_uri': '300379660'},
 '300016593': {'lang': 'en',
  'prefLabel': 'Berber (culture or style)',
  'altLabels': ['Berbers (cultures or styles)'],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': None,
  'aat_uri': '300016593'},
 '300435249': {'lang': 'en',
  'prefLabel': 'Black (general, race and ethnicity)',
  'altLabels': [''],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': 'General designation for the race, ethnicity, styles, culture, and heritage of people of sub-Saharan African origin or descent in places outside Africa, or of member